#WELCOME
In this Assignment, you'll use **Classifier** to predict the output. You are allowed to write/edit between the
```# START CODE HERE``` & ```# END CODE HERE``` only.<br>
Don't edit the test cells otherwise you'd be failed in the assignment.

Upload the ```utils.zip``` file before running the 1st cell.
[Demo](https://drive.google.com/file/d/1ebkQR6ikjGottmxa5KL83ZWdjTLvRxbm/view?usp=share_link)

**BEST OF LUCK**

**RUN THIS CODE TO UNIZP THE REQUIRED FILES**

In [ ]:
!unzip -j /content/utils.zip -d.

Archive:  /content/utils.zip
  inflating: ./weather_classification_data.csv  
  inflating: ./data_scaled_labeled_check.csv  


# Dataset Loading and Pre-Processing

In [ ]:
import pandas as pd
data = pd.read_csv("/content/weather_classification_data.csv")

## Data Info
```data.info``` gives info about data-types and number of null objects.  
There are no null data, no need to remove any.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13200 entries, 0 to 13199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Temperature           13200 non-null  float64
 1   Humidity              13200 non-null  int64  
 2   Wind Speed            13200 non-null  float64
 3   Precipitation (%)     13200 non-null  float64
 4   Cloud Cover           13200 non-null  object 
 5   Atmospheric Pressure  13200 non-null  float64
 6   UV Index              13200 non-null  int64  
 7   Season                13200 non-null  object 
 8   Visibility (km)       13200 non-null  float64
 9   Location              13200 non-null  object 
 10  Weather Type          13200 non-null  object 
dtypes: float64(5), int64(2), object(4)
memory usage: 1.1+ MB


## Data Shuffling
Data Shuffling removes any bias present and creates uniform distribution.

In [ ]:
from sklearn.utils import shuffle
data = shuffle(data)
data.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
6892,31.0,74,14.5,59.0,overcast,1015.54,2,Autumn,5.0,mountain,Rainy
8007,21.0,66,1.0,46.0,overcast,1007.59,4,Autumn,7.0,coastal,Cloudy
7420,65.0,76,7.5,85.0,partly cloudy,1018.49,1,Spring,8.5,coastal,Sunny
3522,29.0,44,4.0,19.0,clear,1023.01,10,Winter,9.5,inland,Sunny
12153,-20.0,59,12.5,64.0,cloudy,1143.28,13,Spring,2.5,coastal,Cloudy


## Scaling and Labeling
Machine can't understand string so all the object datatypes need to be LABELED with integers.<br><br>
Why SCALING?<br>
You might have noticed one column contains data with large values (of order 1000) whereas some column contain small value data (of order 10). As a result, large value columns get more power/importance than others. To remove this disparity we scaled them to same order/values.

**CODE**

We need to find the columns with object datatypes.

HINT: Use ```data.select_dtypes().columns.tolist()```<br>Reference: [w3schools](https://www.w3schools.com/python/pandas/ref_df_select_dtypes.asp#:~:text=The%20select_dtypes()%20method%20returns,the%20specified%20dtype(s).&text=Note%3A%20You%20must%20specify%20at,you%20will%20get%20an%20error.)

In [ ]:
# START CODE HERE
object_columns = data.select_dtypes(include=['object']).columns.tolist()
non_object_columns = data.select_dtypes(exclude=['object']).columns.tolist()
# END CODE HERE
print(f"Object Columns: {object_columns}\nNon Object Columns: {non_object_columns}")

Object Columns: ['Cloud Cover', 'Season', 'Location', 'Weather Type']
Non Object Columns: ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']


**TEST** (DON'T EDIT THIS CELL)

In [ ]:
# DON'T EDIT THIS CELL
if (object_columns == ['Cloud Cover', 'Season', 'Location', 'Weather Type'] and non_object_columns == ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


**CODE**

We need to convert the object-type columns data into integral labels and scale the non-object-type columns data.

HINT: Use ```ColumnTransformer()```<br>Reference: [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), [ChatGPT](https://chatgpt.com/c/7f3c1b02-5da0-4c34-bdb2-d999dccb9c2f)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# START CODE HERE


column_transformer = ColumnTransformer([
    ('encoder', OrdinalEncoder(), object_columns),
    ('scaler', StandardScaler(), non_object_columns)
])

data_scaled_labeled = column_transformer.fit_transform(data)

# Fit and transform the data
data_scaled_labeled = column_transformer.fit_transform(data)
encoder = column_transformer.named_transformers_['encoder']
weather_type_mapping = encoder.categories_[0]


# END CODE HERE

all_columns = object_columns + non_object_columns

# Create the new DataFrame
data_scaled_labeled = pd.DataFrame(data_scaled_labeled, columns=all_columns)

In [ ]:
weather_type_mapping

array(['clear', 'cloudy', 'overcast', 'partly cloudy'], dtype=object)

**TEST** (DON'T EDIT THIS CELL)

In [ ]:
# DON'T EDIT THIS CELL

data_scaled_labeled_check = pd.read_csv("/content/data_scaled_labeled_check.csv")
import numpy as np
data_scaled_labeled_values = np.sort(data_scaled_labeled.values,axis=0)
data_scaled_labeled_check_values = np.sort(data_scaled_labeled_check.values,axis=0)
if np.allclose(data_scaled_labeled_values, data_scaled_labeled_check_values, equal_nan=True):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


## Data Splitting

**CODE**

We need to drop the Weather Type column from data_scaled_labeled to get the X and pick Weather Type column to get Y

Use 90-10 split of data, i.e. 90% for training, 10% for testing

In [ ]:
X = data_scaled_labeled.drop('Weather Type',axis=1)
y = data_scaled_labeled['Weather Type']

# START CODE HERE

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.9, random_state = 10)

**TEST** (DON'T EDIT THIS CELL)

In [ ]:
# DON'T EDIT THIS CELL

if (len(X_train)==11880 and len(X_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

if (len(y_train)==11880 and len(y_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed
Test Passed


In [ ]:
X_train.head()

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
7017,1.0,1.0,0.0,-0.179894,-0.778015,-0.627087,1.294574,-0.164468,-0.520104,2.235614
12278,2.0,3.0,0.0,0.567848,0.063841,-1.061339,-0.834062,0.318889,-0.520104,0.604230
2859,0.0,0.0,1.0,0.682886,-2.412206,-0.192836,-1.428828,0.599011,0.517118,1.049153
5495,0.0,0.0,0.0,2.523482,1.004738,0.892792,0.918933,0.337170,0.517118,-0.730539
8469,0.0,1.0,0.0,0.395293,-1.273224,-1.061339,-1.241007,0.488521,1.295035,0.159307


# Training and Testing
Use any suitable classifier to fit the training data and check the accuracy on test data

Don't forget to import that library

**CODE**

ACCURACY should be more than 90%,

1. Fit the model to X_train and y_train
2. Find the model score on X_test and y_test

In [ ]:
# START CODE HERE

from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(4, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10)



# model.fit(None,None)
loss, score = model.evaluate(X_test,y_test)

# END CODE HERE

print("Accuracy:", score*100)

Epoch 1/10
372/372 [==============================] - 1s 2ms/step - loss: 0.5387 - accuracy: 0.8404
Epoch 2/10
372/372 [==============================] - 1s 2ms/step - loss: 0.3176 - accuracy: 0.8936
Epoch 3/10
372/372 [==============================] - 1s 2ms/step - loss: 0.2773 - accuracy: 0.9003
Epoch 4/10
372/372 [==============================] - 1s 3ms/step - loss: 0.2545 - accuracy: 0.9027
Epoch 5/10
372/372 [==============================] - 1s 3ms/step - loss: 0.2400 - accuracy: 0.9056
Epoch 6/10
372/372 [==============================] - 1s 3ms/step - loss: 0.2286 - accuracy: 0.9088
Epoch 7/10
372/372 [==============================] - 1s 3ms/step - loss: 0.2206 - accuracy: 0.9102
Epoch 8/10
372/372 [==============================] - 1s 2ms/step - loss: 0.2132 - accuracy: 0.9135
Epoch 9/10
372/372 [==============================] - 1s 2ms/step - loss: 0.2072 - accuracy: 0.9135
Epoch 10/10
42/42 [==============================] - 0s 2ms/step - loss: 0.2110 - accuracy: 0.9167
A

**TEST** (DON'T EDIT THIS CELL)

In [ ]:
# DON'T EDIT THIS CELL

if (score>0.90):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


# Predicting Output (Here, Weather Type) (Optional)

In [ ]:
input = X_test.head()
input

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
11923,2.0,3.0,2.0,-0.007338,1.450427,0.169040,1.357181,0.093341,-1.038715,-0.878846
2331,3.0,0.0,2.0,-0.525006,0.063841,1.037543,0.105042,-0.292969,-0.260799,-0.137308
217,2.0,3.0,1.0,-1.100192,-0.233285,-0.771838,-0.051475,-0.570939,-0.779410,-1.323769
1760,0.0,2.0,0.0,0.395293,-2.214122,-0.988963,-1.147097,0.112428,0.517118,0.010999
3963,2.0,3.0,1.0,-1.042674,0.063841,1.399419,1.231967,-0.280602,-1.038715,-0.582231


In [ ]:
y = model.predict(input)
y

1/1 [==============================] - 0s 58ms/step


array([[2.8378081e-03, 9.8321182e-01, 1.0737501e-03, 1.2876599e-02],
       [2.5405201e-01, 7.2422445e-01, 8.9452639e-03, 1.2778244e-02],
       [1.1151251e-03, 2.5259985e-03, 9.9259019e-01, 3.7687283e-03],
       [4.2578264e-04, 8.8626205e-04, 7.1553950e-04, 9.9797243e-01],
       [1.5204232e-04, 3.7002584e-03, 9.9544585e-01, 7.0187799e-04]],
      dtype=float32)

Here, the outputs are in numbers. We can perform inverse column transfer for getiing the string values.


In [ ]:
pred = weather_type_mapping[np.argmax(y, axis=1)]
pred

array(['cloudy', 'cloudy', 'overcast', 'partly cloudy', 'overcast'],
      dtype=object)